In [10]:
import os
from pathlib import Path

import pandas as pd
import rasterio as rio
import rasterio.mask as rio_mask

from nad.load import (
    get_census_level,
    load_ageb_geometry,
    load_block_geometry,
    load_census,
)

In [16]:
data_path = Path(os.environ["DATA_PATH"])
geostatistical_framework_path = Path(os.environ["GEOSTATISTICAL_FRAMEWORK_PATH"])
census_path = Path(os.environ["CENSUS_PATH"])
population_grids_path = Path(os.environ["POPULATION_GRIDS_PATH"])
results_path = Path("./results")

# Load

## AGEBs

In [7]:
df_geom_agebs = load_ageb_geometry(
    population_grids_path / "final" / "zone_agebs" / "shaped" / "2020" / "08.2.03.gpkg",
)

## Manzanas

In [8]:
df_census_base = load_census(census_path / "2020" / "08.csv")
df_census_blocks = get_census_level(df_census_base, level="block")

df_geom_blocks = load_block_geometry(
    geostatistical_framework_path / "2020" / "08_chihuahua" / "08m.shp",
    ageb_cvegeos=df_geom_agebs.index,
)

In [9]:
is_flooded = {}

with rio.open(
    data_path / "datos" / "CdJuarez_60min" / "CdJuarez_60Min_Reclass.tif",
    "r",
) as ds:
    df_reprojected = df_geom_blocks.to_crs(ds.crs)

    for idx, geom in df_reprojected["geometry"].items():
        masked, _ = rio_mask.mask(ds, [geom], crop=True)
        is_flooded[idx] = (masked > 5).any()

In [17]:
pd.Series(is_flooded, name="is_flooded").to_frame().rename_axis(
    "CVEGEO",
    axis=0,
).to_csv(results_path / "flooded_blocks.csv")